In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Arsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [2]:
data=pd.read_csv("C:/Users/Arsh/Desktop/my project/spam text-classification/SPAM text message .csv")

data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
data["Message"][2]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

# EDA

In [6]:
data.isna().sum()

Category    0
Message     0
dtype: int64

In [7]:
data["Category"].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [8]:
data.duplicated().sum()

415

In [9]:
data1=data.copy()

In [10]:
data1.drop_duplicates(keep="first",inplace=True)

data1

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [11]:
data1["Category"].value_counts()

ham     4516
spam     641
Name: Category, dtype: int64

In [12]:
data1

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


# Label Encoding

In [13]:
data1["Category"].unique()

array(['ham', 'spam'], dtype=object)

In [14]:
for cat in data1["Category"]:
    
    if cat=='ham':
        
        data1.loc[data1["Category"]==cat,"Category"]=0
        
    
    elif cat=='spam':
        
        data1.loc[data1["Category"]==cat,"Category"]=1
        
        

In [15]:
data1

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will ü b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


# Text Processing

In [6]:
port_stem=PorterStemmer()

In [17]:
def stemming(content):
    
    stemming_content=re.sub("[^a-zA-Z]"," ",content)
    stemming_content=stemming_content.lower()
    stemming_content=stemming_content.split()
    stemming_content=[port_stem.stem(word) for word in stemming_content if not word in stopwords.words("english")]
    stemming_content=" ".join(stemming_content)
    
    return stemming_content

In [18]:
data1["Message"]=data1["Message"].apply(stemming)

In [19]:
data1["Message"]=data1["Message"].str.strip()

data1

,Category,Message
0,0,go jurong point crazi avail bugi n great world...
1,0,ok lar joke wif u oni
2,1,free entri wkli comp win fa cup final tkt st m...
3,0,u dun say earli hor u c alreadi say
4,0,nah think goe usf live around though
...,...,...
5567,1,nd time tri contact u u pound prize claim easi...
5568,0,b go esplanad fr home
5569,0,piti mood suggest
5570,0,guy bitch act like interest buy someth els nex...


# Balancing Dataset

In [20]:
#! pip install transformers

In [21]:
#! pip install nlpaug

In [22]:
import nlpaug.augmenter.word.context_word_embs as aug

In [23]:
#!pip install torch

In [24]:
augmentor=aug.ContextualWordEmbsAug(model_path="bert-base-uncased",action="insert")

In [25]:
import torch


# Testing Augmentor

In [26]:
text=data1["Message"][2]

text

'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli'

In [27]:
aug_text=augmentor.augment(text)

aug_text=" ".join(aug_text)

print(aug_text)

a free word entri letter wkli comp win fa charity cup knockout final tkt st d may text fa cup receiv message entri question letter std use txt rate c appli


# Augmenting Data

In [28]:
from tqdm import tqdm
from sklearn.utils import shuffle

In [29]:
def aug_data(df,augmentor,repeat=1,samples=200):
    
    aug_text=[]
    
    spam=df[df["Category"]==1].reset_index(drop=True)
    
    for i in tqdm(np.random.randint(0,len(spam),samples)):
        
        for _ in range(repeat):
            
            a_text=augmentor.augment(spam["Message"].iloc[i])
            a_text=" ".join(a_text)
            aug_text.append(a_text)
    
    data={
        
        "Category":1,
        "Message":aug_text        
        
    }
    
    
    aug_df=pd.DataFrame(data)
    
    df=shuffle(df.append(aug_df).reset_index(drop=True))
    
    
    return df

In [30]:
aug_df=aug_data(data1,augmentor,samples=900)

100%|██████████████████████████████████████████████████████████████████████████████| 900/900 [1:55:16<00:00,  7.69s/it]


In [31]:
aug_df=aug_df.reset_index(drop=True)

aug_df

,Category,Message
0,0,also sir sent email log usc payment portal sen...
1,0,say die man feel sad go want know would probab...
2,0,pete pleas ring meiv hardli gotani credit
3,0,oki go yan jiu skip ard oso go cine den go mrt...
4,0,jokin lar depend phone father get lor
...,...,...
6052,0,sure tri make xavier smoke want smoke told smo...
6053,1,email free messag forum activ free text exclus...
6054,0,hiya probabl come home weekend next
6055,0,mmm fuck merri christma


In [32]:
aug_df["Category"].value_counts()

0    4516
1    1541
Name: Category, dtype: int64

In [33]:
aug_df[(aug_df["Category"]==1)]

,Category,Message
5,1,users want new windows nokia colour phone deli...
7,1,dear voucher holder can claim all week offer p...
8,1,just take part search mobil information survey...
12,1,goldvik invit friend repli ye see www sm ac u ...
13,1,winner valu home network custom select receive...
...,...,...
6038,1,in get lion up england tone repli i lionm pro ...
6045,1,urgent alert mobil xxxxxxxxx td bonu award cal...
6046,1,tb persolvo chase us sinc sept definit pay tha...
6050,1,smsservic yourinclus text message credit pl or...


In [34]:
type(aug_df["Message"])

pandas.core.series.Series

In [35]:
aug_df

,Category,Message
0,0,also sir sent email log usc payment portal sen...
1,0,say die man feel sad go want know would probab...
2,0,pete pleas ring meiv hardli gotani credit
3,0,oki go yan jiu skip ard oso go cine den go mrt...
4,0,jokin lar depend phone father get lor
...,...,...
6052,0,sure tri make xavier smoke want smoke told smo...
6053,1,email free messag forum activ free text exclus...
6054,0,hiya probabl come home weekend next
6055,0,mmm fuck merri christma


In [36]:
aug_df.dtypes

Category    object
Message     object
dtype: object

In [37]:
type(aug_df["Message"][5842])



str

In [38]:
corpus=[]

for text in aug_df["Message"]:
    
    corpus.append(text)

    

# Data Preperation

In [39]:
tfidf=TfidfVectorizer()

tfidf.fit(corpus)

x=tfidf.transform(corpus)

x_con=x.toarray()

x_con.shape

(6057, 6912)

In [40]:
tfidf

TfidfVectorizer()

In [41]:
y=aug_df["Category"]

y=y.astype("int")

y.shape

(6057,)

In [42]:
type(y)

pandas.core.series.Series

In [43]:
aug_df

,Category,Message
0,0,also sir sent email log usc payment portal sen...
1,0,say die man feel sad go want know would probab...
2,0,pete pleas ring meiv hardli gotani credit
3,0,oki go yan jiu skip ard oso go cine den go mrt...
4,0,jokin lar depend phone father get lor
...,...,...
6052,0,sure tri make xavier smoke want smoke told smo...
6053,1,email free messag forum activ free text exclus...
6054,0,hiya probabl come home weekend next
6055,0,mmm fuck merri christma


# Model Training

In [44]:
#from sklearn.preprocessing import StandardScaler

In [45]:
#sc=StandardScaler()

#x=sc.fit_transform(x)

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
x_train,x_test,y_train,y_test=train_test_split(x_con,y,test_size=0.20,random_state=2)

In [48]:
x_train.shape

(4845, 6912)

In [49]:
y_train.shape

y_train

5982    0
101     0
5285    0
3963    0
140     1
       ..
1099    0
2514    0
3606    0
5704    0
2575    0
Name: Category, Length: 4845, dtype: int32

In [50]:
x_test.shape

(1212, 6912)

In [51]:
y_test.shape

y_test

4782    0
968     0
5983    1
5555    0
617     0
       ..
2093    0
4457    1
185     1
5842    1
1527    0
Name: Category, Length: 1212, dtype: int32

In [52]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

# Logistic Regression

In [53]:
from sklearn.linear_model import LogisticRegression

In [54]:
logreg=LogisticRegression()

In [55]:
logreg.fit(x_train,y_train)

LogisticRegression()

In [56]:
Y_train_pred=logreg.predict(x_train)

In [57]:
len(Y_train_pred)


4845

In [58]:
len(y_train)

4845

In [59]:
accuracy_score(y_train,Y_train_pred)

0.9785345717234262

In [60]:
confusion_matrix(y_train,Y_train_pred)

array([[3598,   10],
       [  94, 1143]], dtype=int64)

In [61]:
print(classification_report(y_train,Y_train_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      3608
           1       0.99      0.92      0.96      1237

    accuracy                           0.98      4845
   macro avg       0.98      0.96      0.97      4845
weighted avg       0.98      0.98      0.98      4845



In [62]:
y_test_pred=logreg.predict(x_test)

accuracy_score(y_test,y_test_pred)

0.9711221122112211

In [63]:
confusion_matrix(y_test,y_test_pred)

array([[907,   1],
       [ 34, 270]], dtype=int64)

In [64]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       908
           1       1.00      0.89      0.94       304

    accuracy                           0.97      1212
   macro avg       0.98      0.94      0.96      1212
weighted avg       0.97      0.97      0.97      1212



# Logistic Regression is workng  pretty much well on the  Dataset

# Naive Bayes

In [65]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [66]:
gnb=GaussianNB()
mnb=MultinomialNB()
bnb=BernoulliNB()

# GaussianNB

In [67]:
gnb.fit(x_train,y_train)

GaussianNB()

In [68]:
Y_train_pred=gnb.predict(x_train)

In [69]:
accuracy_score(y_train,Y_train_pred)

0.9050567595459237

In [70]:
confusion_matrix(y_train,Y_train_pred)

array([[3148,  460],
       [   0, 1237]], dtype=int64)

In [71]:
print(classification_report(y_train,Y_train_pred))

              precision    recall  f1-score   support

           0       1.00      0.87      0.93      3608
           1       0.73      1.00      0.84      1237

    accuracy                           0.91      4845
   macro avg       0.86      0.94      0.89      4845
weighted avg       0.93      0.91      0.91      4845



In [72]:
y_test_pred=gnb.predict(x_test)

accuracy_score(y_test,y_test_pred)

0.8605610561056105

In [73]:
confusion_matrix(y_test,y_test_pred)

array([[744, 164],
       [  5, 299]], dtype=int64)

In [74]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.99      0.82      0.90       908
           1       0.65      0.98      0.78       304

    accuracy                           0.86      1212
   macro avg       0.82      0.90      0.84      1212
weighted avg       0.91      0.86      0.87      1212



# Gaussian Naive Bayes' performance is not so good as Logistic Regression

# MultinomialNB

In [75]:
mnb.fit(x_train,y_train)

MultinomialNB()

In [76]:
Y_train_pred=mnb.predict(x_train)

In [77]:
accuracy_score(y_train,Y_train_pred)

0.9869969040247678

In [78]:
confusion_matrix(y_train,Y_train_pred)

array([[3599,    9],
       [  54, 1183]], dtype=int64)

In [79]:
print(classification_report(y_train,Y_train_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      3608
           1       0.99      0.96      0.97      1237

    accuracy                           0.99      4845
   macro avg       0.99      0.98      0.98      4845
weighted avg       0.99      0.99      0.99      4845



In [80]:
y_test_pred=mnb.predict(x_test)

accuracy_score(y_test,y_test_pred)

0.9826732673267327

In [81]:
confusion_matrix(y_test,y_test_pred)

array([[906,   2],
       [ 19, 285]], dtype=int64)

In [82]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       908
           1       0.99      0.94      0.96       304

    accuracy                           0.98      1212
   macro avg       0.99      0.97      0.98      1212
weighted avg       0.98      0.98      0.98      1212



# LSvc  also have a very good performance , however MultinomialNB has slightly better performance

# MultinomiaNB have the best performance as compared to other  models

# BernouliNB

In [83]:
bnb.fit(x_train,y_train)

BernoulliNB()

In [84]:
Y_train_pred=mnb.predict(x_train)

In [85]:
accuracy_score(y_train,Y_train_pred)

0.9869969040247678

In [86]:
confusion_matrix(y_train,Y_train_pred)

array([[3599,    9],
       [  54, 1183]], dtype=int64)

In [87]:
print(classification_report(y_train,Y_train_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      3608
           1       0.99      0.96      0.97      1237

    accuracy                           0.99      4845
   macro avg       0.99      0.98      0.98      4845
weighted avg       0.99      0.99      0.99      4845



In [88]:
y_test_pred=bnb.predict(x_test)

accuracy_score(y_test,y_test_pred)

0.9777227722772277

In [89]:
confusion_matrix(y_test,y_test_pred)

array([[905,   3],
       [ 24, 280]], dtype=int64)

In [90]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       908
           1       0.99      0.92      0.95       304

    accuracy                           0.98      1212
   macro avg       0.98      0.96      0.97      1212
weighted avg       0.98      0.98      0.98      1212



# Not much difference between Bernouli and Multinomial NB, however precision of Multinomial NB in test dataset is better 

# SVC

In [91]:
from sklearn.svm import LinearSVC

In [92]:
lsvc=LinearSVC()

lsvc.fit(x_train,y_train)

LinearSVC()

In [93]:
Y_train_pred=lsvc.predict(x_train)

In [94]:
accuracy_score(y_train,Y_train_pred)

0.9991744066047472

In [130]:
y_train.shape

(4845,)

In [95]:
confusion_matrix(y_train,Y_train_pred)

array([[3606,    2],
       [   2, 1235]], dtype=int64)

In [96]:
print(classification_report(y_train,Y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3608
           1       1.00      1.00      1.00      1237

    accuracy                           1.00      4845
   macro avg       1.00      1.00      1.00      4845
weighted avg       1.00      1.00      1.00      4845



In [97]:
y_test_pred=bnb.predict(x_test)

accuracy_score(y_test,y_test_pred)

0.9777227722772277

In [98]:
confusion_matrix(y_train,Y_train_pred)

array([[3606,    2],
       [   2, 1235]], dtype=int64)

In [99]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       908
           1       0.99      0.92      0.95       304

    accuracy                           0.98      1212
   macro avg       0.98      0.96      0.97      1212
weighted avg       0.98      0.98      0.98      1212



# We are going in for Multinomial NB

# Testing Model with random samples

In [100]:
real=data1[(data1["Category"]==0)].head()
fake=data1[(data1["Category"]==1)].head()

In [101]:
real

,Category,Message
0,0,go jurong point crazi avail bugi n great world...
1,0,ok lar joke wif u oni
3,0,u dun say earli hor u c alreadi say
4,0,nah think goe usf live around though
6,0,even brother like speak treat like aid patent


In [102]:
fake

,Category,Message
2,1,free entri wkli comp win fa cup final tkt st m...
5,1,freemsg hey darl week word back like fun still...
8,1,winner valu network custom select receivea pri...
9,1,mobil month u r entitl updat latest colour mob...
11,1,six chanc win cash pound txt csh send cost p d...


In [103]:
sample=pd.concat([real,fake])

sample
                  

,Category,Message
0,0,go jurong point crazi avail bugi n great world...
1,0,ok lar joke wif u oni
3,0,u dun say earli hor u c alreadi say
4,0,nah think goe usf live around though
6,0,even brother like speak treat like aid patent
2,1,free entri wkli comp win fa cup final tkt st m...
5,1,freemsg hey darl week word back like fun still...
8,1,winner valu network custom select receivea pri...
9,1,mobil month u r entitl updat latest colour mob...
11,1,six chanc win cash pound txt csh send cost p d...


In [108]:
corpus1

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'even brother like speak treat like aid patent',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info']

In [114]:
y=sample["Category"]

y

0     0
1     0
3     0
4     0
6     0
2     1
5     1
8     1
9     1
11    1
Name: Category, dtype: object

In [118]:
x1=tfidf.transform(corpus1)

x1.shape

(10, 6912)

In [119]:
y_pred=mnb.predict(x1)

y_pred.shape

(10,)

In [122]:
y.shape

(10,)

In [123]:
right=0
wrong=0

#y_pred=y_pred.tolist()
#y=y.tolist()

for y,yp in zip(y,y_pred):
    
    if y==yp:
        
        right=right+1
    
    else:
        
        wrong=wrong+1

print(right)
print(wrong)

9
1


In [136]:
print("Righty predicted ans",(right/10)*100,"%")
print("Wrongly predicted ans",(wrong/10)*100,"%")

Righty predicted ans 90.0 %
Wrongly predicted ans 10.0 %


#  Model is working good going ahead to pickle the model and vectorizer

# Pickling

In [137]:
import joblib



# Vectorizer

In [138]:
joblib.dump(tfidf,"C:/Users/Arsh/Desktop/my project/spam text-classification/vectorizer.pkl")

['C:/Users/Arsh/Desktop/my project/spam text-classification/vectorizer.pkl']

# Multinomial NB

In [139]:
joblib.dump(mnb,"C:/Users/Arsh/Desktop/my project/spam text-classification/mnb_classifier.pkl")

['C:/Users/Arsh/Desktop/my project/spam text-classification/mnb_classifier.pkl']

In [12]:
def stemming(content):
    
    stemming_content=re.sub("[^a-zA-Z]"," ",content)
    stemming_content=stemming_content.lower()
    stemming_content=stemming_content.split()
    stemming_content=[port_stem.stem(word) for word in stemming_content if not word in stopwords.words("english")]
    stemming_content=" ".join(stemming_content)
    
    return stemming_content

In [25]:
msg="Amazon is sending you with a refund of $100 please reply with your bank account and IFSC code to recive refund "

stem_content=stemming(msg)

stem_content

'amazon send refund pleas repli bank account ifsc code reciv refund'

In [9]:
import joblib
tfidf=joblib.load("C:/Users/Arsh/Desktop/my project/spam text-classification/vectorizer.pkl")
mnb=joblib.load("C:/Users/Arsh/Desktop/my project/spam text-classification/mnb_classifier.pkl")

In [26]:
corpus=[stem_content]

x=tfidf.transform(corpus)

x.shape

(1, 6912)

In [10]:
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
port_stem=PorterStemmer()

In [33]:
TextBlob(msg).sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [46]:
def stemming(content):
    
    stemming_content=re.sub("[^a-zA-Z]"," ",content)
    stemming_content=stemming_content.lower()
    visit=msg.find("visit")
    if visit!=1:
        
        fmsg=msg[:visit]+"https:"+msg[visit+5:]
        stemming_content=fmsg   
       
    stemming_content=stemming_content.split()
    stemming_content=[port_stem.stem(word) for word in stemming_content if not word in stopwords.words("english")]
    stemming_content=" ".join(stemming_content)
    
    return stemming_content

In [47]:
msg="You have recieved $1,000 from Walmart gift card , visit //google/nfg8855"
stem_content=stemming(msg)

print(stem_content)

you reciev $1,000 walmart gift card , https: //google/nfg8855


In [49]:
msg="You have recieved $1,000 from Walmart gift card , visit  //google/nfg8855"

stem_content=stemming(msg)

corpus=[stem_content]

x=tfidf.transform(corpus)

y_pred=mnb.predict(x)

res=y_pred[0]

if res==0:
    
    print("Real")
else:
    
    print("Fake")

    
subject=TextBlob(msg).sentiment.subjectivity
polarity=TextBlob(msg).sentiment.polarity

if polarity>0:
    
    polarity="Positive"

elif  polarity==0:
    
    polarity="Neutral"

elif  polarity<0:
    
    polarity="Negative"    
    
    
print(polarity)
print(subject)



Fake
Neutral
0.0


In [32]:
msg="You have recieved $1,000 from Walmart gift card , visit google/nfg8855"

visit=msg.find("visit")

fmsg=msg[visit+5]+"https:"+msg[visit+5:]
print(fmsg)

 https: google/nfg8855


In [51]:
fmsg.replace("/","")

' https: googlenfg8855'

In [38]:
if visit!=-1:
    
    print("visit yes")
else:
    print("no visit")

visit yes


In [48]:
msg[visit:]+"test"+msg[visit+5:]

'visit //google/nfg8855test //google/nfg8855'

In [55]:
msg1="You have recieved $1,000 from Walmart gift card , visit https:google/nfg8855"

msg2=msg1.replace("https:","").replace("http:","").replace("/","").replace(r"\"","")
                                                                      

#msg1.replace("h:","")
msg2

'You have recieved $1,000 from Walmart gift card , visit googlenfg8855'